In [2]:
%matplotlib widget

In [1]:
import numpy as np
import pandas as pd
import random

from tensorflow import keras as K

In [3]:
data = pd.read_hdf('../feature_exploration/features.h5', key='no_preprocessing')

In [4]:
# get the subjects for which LOSO actually makes sense: those with multiple activities (ie more than just walking)
gbc = data.groupby(['Subject', 'Activity'], as_index=False).count()
loso_subjects = [i for i in gbc.Subject.unique() if gbc.loc[gbc.Subject == i].shape[0] > 3]

random.seed(5)  # fix the generation so that its the same every time
random.shuffle(loso_subjects)

training_masks = []
validation_masks = []
testing_masks = []

for i in range(0, len(loso_subjects), 3):
    tr_m = np.ones(data.shape[0], dtype='bool')
    v_m = np.zeros(data.shape[0], dtype='bool')
    
    for j in range(3):
        tr_m &= data.Subject != loso_subjects[i+j]
    for j in range(2):
        v_m |= data.Subject == loso_subjects[i+j]
    te_m = data.Subject == loso_subjects[i+2]
    
    training_masks.append(tr_m)
    validation_masks.append(v_m)
    testing_masks.append(te_m)

masks = (training_masks, validation_masks, testing_masks)

In [20]:
model = K.Sequential([
    K.layers.Input(shape=(26,)),
    K.layers.Dense(32, activation='relu'),
    K.layers.Dense(16, activation='relu'),
    K.layers.Dense(8, activation='relu'),
    K.layers.Dense(1, activation='softmax')
])

In [27]:
model.compile(
    optimizer=K.optimizers.SGD(learning_rate=.1),
    loss=K.losses.MeanSquaredError(),
    metrics=K.metrics.Accuracy()
)

In [28]:
x_train = data.loc[training_masks[0]].iloc[:, 3:]
y_train = data.loc[training_masks[0], 'Label']

In [29]:
model.fit(
    x_train,
    y_train,
    batch_size=128,
    epochs=15
)

Epoch 1/15
1096/1096 [==============================] - 2s 2ms/step - loss: 0.7036 - accuracy: 0.2964
Epoch 2/15
1096/1096 [==============================] - 3s 2ms/step - loss: 0.7036 - accuracy: 0.2964
Epoch 3/15
1096/1096 [==============================] - 2s 2ms/step - loss: 0.7036 - accuracy: 0.2964
Epoch 4/15
1096/1096 [==============================] - 2s 2ms/step - loss: 0.7036 - accuracy: 0.2964
Epoch 5/15
1096/1096 [==============================] - 3s 2ms/step - loss: 0.7036 - accuracy: 0.2964
Epoch 6/15
1096/1096 [==============================] - 3s 2ms/step - loss: 0.7036 - accuracy: 0.2964
Epoch 7/15
1096/1096 [==============================] - 2s 2ms/step - loss: 0.7036 - accuracy: 0.2964
Epoch 8/15
1096/1096 [==============================] - 3s 2ms/step - loss: 0.7036 - accuracy: 0.2964
Epoch 9/15
1096/1096 [==============================] - 3s 2ms/step - loss: 0.7036 - accuracy: 0.2964
Epoch 10/15
1096/1096 [==============================] - 3s 2ms/step - loss: 0.703